In [1]:
import os
import shutil


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import os
import shutil

# Define the paths
tomato_folder = '/content/drive/MyDrive/New Plant Diseases Dataset(Augmented)/train'
cotton_folder = '/content/drive/MyDrive/cotton_files/cotton'
new_folder = '/content/drive/MyDrive/new_data'

# Create the new folder if it doesn't exist
os.makedirs(new_folder, exist_ok=True)

def copy_directory(src_folder, dest_folder):
    """
    Recursively copy the contents of the src_folder to dest_folder.
    """
    for item in os.listdir(src_folder):
        src_path = os.path.join(src_folder, item)
        dest_path = os.path.join(dest_folder, item)

        if os.path.isdir(src_path):
            # If it's a directory, create a corresponding directory in the destination
            if not os.path.exists(dest_path):
                os.makedirs(dest_path)
            # Recursively copy the contents of the directory
            copy_directory(src_path, dest_path)
        else:
            # If it's a file, copy it
            shutil.copy(src_path, dest_path)

def combine_folders(folder1, folder2, dest_folder):
    """
    Combine contents of folder1 and folder2 into dest_folder.
    """
    # Copy contents of the first folder
    copy_directory(folder1, dest_folder)

    # Copy contents of the second folder
    copy_directory(folder2, dest_folder)

# Combine folders
combine_folders(tomato_folder, cotton_folder, new_folder)

# Print the contents of the new folder to confirm
print("Contents of the new folder:")
for root, dirs, files in os.walk(new_folder):
    for name in files:
        print(os.path.join(root, name))
    for name in dirs:
        print(os.path.join(root, name))


Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/ce1c325e-c775-4428-92fe-b907fd5eef83___Matt.S_CG 1229.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/da4d940c-2a77-481b-8693-167632534fc8___Matt.S_CG 0858.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/d90c2358-fa7a-4ad8-913f-37ff4cfd29d0___JR_Sept.L.S 8346.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/d7d247bb-ea19-452f-952a-5919d167e33c___Matt.S_CG 0932.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/d645a5d8-a404-4825-8be7-a15725ee26d8___Matt.S_CG 2629.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/cd3b1a93-f5b4-406f-aa8a-640280edf543___Keller.St_CG 1860.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/d2f97435-6763-41b9-9c83-44fb45fa4d24___Matt.S_CG 1051.JPG
/content/drive/MyDrive/new_data/Tomato___Septoria_leaf_spot/d47bf62b-98b7-434a-881f-0ad1db20315c___Matt.S_CG 6753.JPG


In [7]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [20]:
# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/new_data"

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),  # Resize images to a consistent size
    batch_size=32,          # Number of images per batch
    label_mode='categorical',  # or 'int' for numeric labels
    shuffle=True            # Shuffle the dataset
)

# Check the dataset
for images, labels in dataset.take(1):
    print(images.shape)  # (batch_size, height, width, channels)
    print(labels.shape)  # (batch_size, number_of_classes)

Found 18616 files belonging to 14 classes.
(32, 128, 128, 3)
(32, 14)


In [21]:
import tensorflow as tf

# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/new_data"

# Load the training dataset (80%)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='training',            # This is the training set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Load the testing dataset (20%)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='validation',          # This is the validation/test set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Check the datasets
for images, labels in train_dataset.take(1):
    print(f"Train: {images.shape}, {labels.shape}")

for images, labels in test_dataset.take(1):
    print(f"Test: {images.shape}, {labels.shape}")

Found 18616 files belonging to 14 classes.
Using 14893 files for training.
Found 18616 files belonging to 14 classes.
Using 3723 files for validation.
Train: (32, 128, 128, 3), (32,)
Test: (32, 128, 128, 3), (32,)


In [ ]:
# Import the required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50

# Create the base model from the pre-trained model ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes in your dataset
predictions = Dense(4, activation='softmax')(x)  # Adjust the number of classes as needed

# This is the model we will train
resnet50_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Import the required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg19 import VGG19


# Create the base model from the pre-trained model VGG19
base_model = VGG19(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes in your dataset
predictions = Dense(4, activation='softmax')(x) # 5 classes

# this is the model we will train
vgg19_model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG19 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Load the InceptionV3 model without the top layer (include_top=False)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128,128, 3))

# Add new layers to the model for our classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.applications import ResNet50, VGG19, InceptionV3
from tensorflow.keras.optimizers import Adam

# Define input shape
input_shape = (128, 128, 3)

# Create base models
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_vgg = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze base model layers
for layer in base_model_resnet.layers:
    layer.trainable = False

for layer in base_model_vgg.layers:
    layer.trainable = False

for layer in base_model_inception.layers:
    layer.trainable = False

# Define input tensor
input_tensor = Input(shape=input_shape)

# Get the outputs of the base models
output_resnet = base_model_resnet(input_tensor)
output_vgg = base_model_vgg(input_tensor)
output_inception = base_model_inception(input_tensor)

# Add global average pooling layers
x_resnet = GlobalAveragePooling2D()(output_resnet)
x_vgg = GlobalAveragePooling2D()(output_vgg)
x_inception = GlobalAveragePooling2D()(output_inception)

# Concatenate features from all models
combined_features = Concatenate()([x_resnet, x_vgg, x_inception])

# Add fully-connected layer
x = Dense(1024, activation='relu')(combined_features)

# Add the output layer
predictions = Dense(14, activation='softmax')(x)  # Adjust the number of classes as needed

# Define the combined model
stacked_model = Model(inputs=input_tensor, outputs=predictions)

# Compile the combined model
stacked_model.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Summary of the model
stacked_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 128, 128, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resnet50 (Functional)     │ (None, 4, 4, 2048)     │     23,587,712 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ vgg19 (Functional)        │ (None, 4, 4, 512)      │     20,024,384 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ inception_v3 (Functional) │ (None, 2, 2, 2048)     │     21,802,784 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ resnet50[0][0]         │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 512)            │              0 │ vgg19[0][0]            │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling2d… │ (None, 2048)           │              0 │ inception_v3[0][0]     │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 4608)           │              0 │ global_average_poolin… │
│ (Concatenate)             │                        │                │ global_average_poolin… │
│                           │                        │                │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1024)           │      4,719,616 │ concatenate_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 14)             │         14,350 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 70,148,846 (267.60 MB)

 Trainable params: 4,733,966 (18.06 MB)

 Non-trainable params: 65,414,880 (249.54 MB)

In [24]:
#Specifing epochs & batch size for vggnet16
epochs = 40
batch_size = 32
image_height = 128
image_width = 128

In [25]:
#Creating an object of ImageDataGenerator for augmenting training dataset
train_datagen = ImageDataGenerator(rescale= 1./255,
rotation_range=10,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.1,
zoom_range=0.1,
horizontal_flip=True,
fill_mode='nearest')

#Creating an object of ImageDataGenerator for augmenting test dataset
test_datagen = ImageDataGenerator(rescale= 1./255)

In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (128, 128)   # VGG19 requires 128*128 input
batch_size = 32           # Number of images to process in a batch

# Define the train data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Apply random shear transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True,      # Randomly flip images horizontally
    validation_split=0.2      # Split 20% of the data for validation
)

# Define the test/validation data generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set dataset directory path
dataset_path = "/content/drive/MyDrive/new_data"

# Create train data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='training',           # Use this for the training data
    shuffle=True                 # Shuffle the data for better training
)

# Create validation data generator
validation_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='validation',         # Use this for the validation data
    shuffle=False                # No need to shuffle the validation data
)

# Check the generators
print(f"Train Generator: {len(train_generator)} batches")
print(f"Validation Generator: {len(validation_generator)} batches")


Found 14896 images belonging to 14 classes.
Found 3720 images belonging to 14 classes.
Train Generator: 466 batches
Validation Generator: 117 batches


In [29]:
# Define image size and batch size
image_size = (128, 128)
batch_size = 32           # Number of images to process in a batch

# Define the test data generator (only rescale, no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Set dataset directory path for test data
test_dataset_path = "/content/drive/MyDrive/new_data"

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    shuffle=False                # Do not shuffle the test data
)

# Check the test generator
print(f"Test Generator: {len(test_generator)} batches")

Found 18616 images belonging to 14 classes.
Test Generator: 582 batches


In [30]:
stacked_model_history = stacked_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


466/466 ━━━━━━━━━━━━━━━━━━━━ 5913s 13s/step - accuracy: 0.5591 - loss: 1.3591 - val_accuracy: 0.7280 - val_loss: 0.8090
Epoch 2/40
466/466 ━━━━━━━━━━━━━━━━━━━━ 5827s 13s/step - accuracy: 0.7636 - loss: 0.7138 - val_accuracy: 0.7339 - val_loss: 0.7985
Epoch 3/40
424/466 ━━━━━━━━━━━━━━━━━━━━ 7:01 10s/step - accuracy: 0.7929 - loss: 0.6185

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report

# predictions from the test data
y_pred = stacked_model.predict(test_generator)

# transform class labels of the predictions
y_pred_classes = np.argmax(y_pred, axis=1)

# Real classes
y_true = test_generator.classes

# Create classification report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# print report
print(report)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 50/582 ━━━━━━━━━━━━━━━━━━━━ 1:27:54 10s/step